In [1]:
import json
import requests
import csv
import re
from lxml import etree

In [2]:
import pandas as pd

In [12]:
acct=pd.read_csv(r'/home/junluo/Documents/stocklist.csv',dtype={'Stocks':str})

In [13]:
acct

,Stocks
0,000002
1,123067
2,123070
3,127018
4,127019
...,...
81,127027
82,123085
83,113569
84,128100


In [14]:
acct['CODE']=acct['Stocks']

In [15]:
import json
import requests
import csv
import re
from lxml import etree
newUrl ="https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1584777951900"
#最简单的爬虫请求.也可以加上headers字段，防止部分网址的反爬虫机制
response = requests.get(newUrl)
#当爬取的界面需要用户名密码登录时候，构建的请求需要包含auth字段
data = response.content.decode("utf-8")
dat = json.loads(data)

tbl=pd.DataFrame([row['cell'] for row in dat['rows']])
tbl['force_redeem']=tbl['force_redeem'].fillna('')
redeem=tbl[(tbl['force_redeem']!='')&(-tbl['force_redeem'].str.contains('不行使'))][['bond_id','bond_nm','force_redeem']
                                                                                ].copy()

In [16]:
redeem=tbl[(tbl['force_redeem']!='')&(-tbl['force_redeem'].str.contains('不行使'))][['bond_id','bond_nm','force_redeem']
                                                                                ].copy()

In [17]:
redeem['bond_nm'].unique()

array(['隆20转债', '明阳转债', '奥瑞转债', '凯龙转债', '红相转债'], dtype=object)

In [18]:
len(redeem['bond_nm'].unique())

5

In [19]:
checkredeem=pd.merge(redeem,acct,
        how='left',
        left_on='bond_id',
        right_on='CODE')
checkredeem[-checkredeem['CODE'].isnull()]

,bond_id,bond_nm,force_redeem,Stocks,CODE


In [20]:
redeem

,bond_id,bond_nm,force_redeem
7,113038,隆20转债,最后交易日：2021年3月30日\r\n最后转股日：2021年3月30日\r\n赎回价：10...
8,113029,明阳转债,最后交易日：2021年3月18日\r\n最后转股日：2021年3月18日\r\n赎回价：10...
36,128096,奥瑞转债,2020年11月18日公告：本次（2020年9月28日至2020年11月16日）不提前赎回奥...
178,128052,凯龙转债,最后交易日：2021年3月23日\r\n最后转股日：2021年3月23日\r\n赎回价：10...
255,123044,红相转债,2020年11月10日公告，本次（2020年10月19日至2020年11月10日）不提前赎回...


In [21]:
tbl[(-tbl['force_redeem'].str.contains('不行使'))&(tbl['force_redeem']!='')][['bond_nm','force_redeem']]

,bond_nm,force_redeem
7,隆20转债,最后交易日：2021年3月30日\r\n最后转股日：2021年3月30日\r\n赎回价：10...
8,明阳转债,最后交易日：2021年3月18日\r\n最后转股日：2021年3月18日\r\n赎回价：10...
36,奥瑞转债,2020年11月18日公告：本次（2020年9月28日至2020年11月16日）不提前赎回奥...
178,凯龙转债,最后交易日：2021年3月23日\r\n最后转股日：2021年3月23日\r\n赎回价：10...
255,红相转债,2020年11月10日公告，本次（2020年10月19日至2020年11月10日）不提前赎回...
